#  Deep Lake

>[Deep Lake](https://docs.activeloop.ai/) as a Multi-Modal Vector Store that stores embeddings and their metadata including text, jsons, images, audio, video, and more. It saves the data locally, in your cloud, or on Activeloop storage. It performs hybrid search including embeddings and their attributes.

This notebook showcases basic functionality related to `Deep Lake`. While `Deep Lake` can store embeddings, it is capable of storing any type of data. It is a fully fledged serverless data lake with version control, query engine and streaming dataloader to deep learning frameworks.  

For more information, please see the Deep Lake [documentation](https://docs.activeloop.ai) or [api reference](https://docs.deeplake.ai)

In [ ]:
!pip install openai deeplake tiktoken

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import DeepLake

Could not import azure.core python package.


In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
embeddings = OpenAIEmbeddings()

In [11]:
from langchain.document_loaders import TextLoader

loader = TextLoader('../../../state_of_the_union.txt', encoding='utf-8') # For windows users specifying the encoding is mandatory
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Create a dataset locally at `./deeplake/`, then run similiarity search. The Deeplake+LangChain integration uses Deep Lake datasets under the hood, so `dataset` and `vector store` are used interchangeably. To create a dataset in your own cloud, or in the Deep Lake storage, [adjust the path accordingly](https://docs.activeloop.ai/storage-and-credentials/storage-options).

In [12]:
db = DeepLake(dataset_path="./my_deeplake/", embedding_function=embeddings)
db.add_documents(docs)
# or shorter
# db = DeepLake.from_documents(docs, dataset_path="./my_deeplake/", embedding=embeddings, overwrite=True)
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

./my_deeplake/ loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:04<00:00
/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


In [6]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


Later, you can reload the dataset without recomputing embeddings

In [13]:
db = DeepLake(dataset_path="./my_deeplake/", embedding_function=embeddings, read_only=True)
docs = db.similarity_search(query)

./my_deeplake/ loaded successfully.



Deep Lake Dataset in ./my_deeplake/ already exists, loading from the storage


/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


Deep Lake, for now, is single writer and multiple reader. Setting `read_only=True` helps to avoid acquring the writer lock.

### Retrieval Question/Answering

In [14]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAIChat

qa = RetrievalQA.from_chain_type(llm=OpenAIChat(model='gpt-3.5-turbo'), chain_type='stuff', retriever=db.as_retriever())

/home/ubuntu/langchain/langchain/llms/openai.py:695: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [15]:
query = 'What did the president say about Ketanji Brown Jackson'
qa.run(query)

/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


'The president nominated Ketanji Brown Jackson to serve on the United States Supreme Court and described her as a former top litigator in private practice, a former federal public defender, a consensus builder, and from a family of public school educators and police officers.'

### Attribute based filtering in metadata

In [16]:
import random

for d in docs:
    d.metadata['year'] = random.randint(2012, 2014)

db = DeepLake.from_documents(docs, embeddings, dataset_path="./my_deeplake/", overwrite=True)

./my_deeplake/ loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:04<00:00


In [17]:
db.similarity_search('What did the president say about Ketanji Brown Jackson', filter={'year': 2013})

100%|██████████| 4/4 [00:00<00:00, 1782.91it/s]
/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt', 'year': 2013}),
 Document(page_content='And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my des

### Choosing distance function
Distance function `L2` for Euclidean, `L1` for Nuclear, `Max` l-infinity distnace, `cos` for cosine similarity, `dot` for dot product 

In [18]:
db.similarity_search('What did the president say about Ketanji Brown Jackson?', distance_metric='cos')

/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt', 'year': 2013}),
 Document(page_content='A former top litigator in private practice. A former federal public defender. And fro

### Maximal Marginal relevance
Using maximal marginal relevance

In [19]:
db.max_marginal_relevance_search('What did the president say about Ketanji Brown Jackson?')

Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)
Using maximal marginal relevance with big datasets is discouraged as it may lead to performance issues.


[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../../state_of_the_union.txt', 'year': 2013}),
 Document(page_content='And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my des

### Delete dataset

In [14]:
db.delete_dataset()

./my_deeplake/ loaded successfully.


and if delete fails you can also force delete

In [15]:
DeepLake.force_delete_by_path("./my_deeplake")

## Deep Lake datasets on cloud (Activeloop, AWS, GCS, etc.) or in memory
By default deep lake datasets are stored locally, in case you want to store them in memory, in the Deep Lake Managed DB, or in any object storage, you can provide the [corresponding path to the dataset](https://docs.activeloop.ai/storage-and-credentials/storage-options). You can retrieve your user token from [app.activeloop.ai](https://app.activeloop.ai/)

In [4]:
os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('Activeloop Token:')

In [63]:
# Embed and store the texts
username = "<username>" # your username on app.activeloop.ai   
dataset_path = f"hub://{username}/langchain_test" # could be also ./local/path (much faster locally), s3://bucket/path/to/dataset, gcs://path/to/dataset, etc.

embedding = OpenAIEmbeddings()
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings, overwrite=True)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/davitbun/langchain_test
hub://davitbun/langchain_test loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:14<00:00


Dataset(path='hub://davitbun/langchain_test', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (4, 1536)  float32   None   
    ids      text     (4, 1)      str     None   
 metadata    json     (4, 1)      str     None   
   text      text     (4, 1)      str     None   


['d6d6ccb4-e187-11ed-b66d-41c5f7b85421',
 'd6d6ccb5-e187-11ed-b66d-41c5f7b85421',
 'd6d6ccb6-e187-11ed-b66d-41c5f7b85421',
 'd6d6ccb7-e187-11ed-b66d-41c5f7b85421']

### Different execution options:

by default deeplake vector store, uses python implementation to search inside of the dataset. However, for the comparitively large datasets (~1-2 millions of rows), it is much better to use another options, as python implementation could be quite slow. So, there are three execution options: `python`, `tensor_db`, `compute_engine`. 
- `python`: runs on the client and can be used for any data stored anywhere. This is used by default. WARNING: using this option with big datasets is discouraged, because it can lead to some memory issues.
- `compute_engine`: runs on the client and can be used for any data stored in or connected to Deep Lake.
- `tensor_db`: runs on the Deep Lake Managed Database and can be used for any data stored in the Deep Lake Managed.

#### python

In [21]:
db = DeepLake(dataset_path='hub://activeloop/twitter-algorithm', embedding_function=embeddings, exec_option="python")

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/twitter-algorithm

hub://activeloop/twitter-algorithm loaded successfully.



Deep Lake Dataset in hub://activeloop/twitter-algorithm already exists, loading from the storage


In [18]:
db.similarity_search("what is twitter?")

/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


Python implementation fetches all of the dataset's embedding into memory. With big datasets this could be quite slow and potentially result in performance issues. Use `exec_option = 'tensor_db'` for better performance.


[Document(page_content='Tweepcred\n\nTweepcred is a social network analysis tool that calculates the influence of Twitter users based on their interactions with other users. The tool uses the PageRank algorithm to rank users based on their influence.\n\nPageRank Algorithm\nPageRank is a graph algorithm that was originally developed by Google to determine the importance of web pages in search results. The algorithm works by assigning a numerical score to each page based on the number and quality of other pages that link to it. The more links a page has from other high-quality pages, the higher its PageRank score.\n\nIn the Tweepcred project, the PageRank algorithm is used to determine the influence of Twitter users based on their interactions with other users. The graph is constructed by treating Twitter users as nodes, and their interactions (mentions, retweets, etc.) as edges. The PageRank score of a user represents their influence in the network.', metadata={'source': './the-algorith

#### tensor_db

In [19]:
db = DeepLake(dataset_path='hub://activeloop/twitter-algorithm', embedding_function=embeddings, exec_option="tensor_db")

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/twitter-algorithm

hub://activeloop/twitter-algorithm loaded successfully.



Deep Lake Dataset in hub://activeloop/twitter-algorithm already exists, loading from the storage


In [20]:
db.similarity_search("what is twitter?")

/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


[Document(page_content='Tweepcred\n\nTweepcred is a social network analysis tool that calculates the influence of Twitter users based on their interactions with other users. The tool uses the PageRank algorithm to rank users based on their influence.\n\nPageRank Algorithm\nPageRank is a graph algorithm that was originally developed by Google to determine the importance of web pages in search results. The algorithm works by assigning a numerical score to each page based on the number and quality of other pages that link to it. The more links a page has from other high-quality pages, the higher its PageRank score.\n\nIn the Tweepcred project, the PageRank algorithm is used to determine the influence of Twitter users based on their interactions with other users. The graph is constructed by treating Twitter users as nodes, and their interactions (mentions, retweets, etc.) as edges. The PageRank score of a user represents their influence in the network.', metadata={'source': './the-algorith

NOTE: to be able to use this feature, when creating the dataset you should specify `exec_option="tensor_db"`. Otherwise it won't work

#### compute_engine

to use this option you need to install enterpise package. `pip install deeplake[enterprise]`

In [ ]:
!pip install deeplake[enterprise]

In [5]:
token = os.getenv("ACTIVELOOP_TOKEN")

In [7]:
db = DeepLake(dataset_path='hub://activeloop/twitter-algorithm', embedding_function=embeddings, exec_option="compute_engine", token=token)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/twitter-algorithm

hub://activeloop/twitter-algorithm loaded successfully.



Deep Lake Dataset in hub://activeloop/twitter-algorithm already exists, loading from the storage


In [8]:
db.similarity_search("what is twitter?")

/home/ubuntu/langchain/langchain_venv/lib/python3.10/site-packages/deeplake/util/warnings.py:7: UserWarning: both embedding and embedding_function are specified. 
  warnings.warn(*args, **kwargs)


[Document(page_content='Tweepcred\n\nTweepcred is a social network analysis tool that calculates the influence of Twitter users based on their interactions with other users. The tool uses the PageRank algorithm to rank users based on their influence.\n\nPageRank Algorithm\nPageRank is a graph algorithm that was originally developed by Google to determine the importance of web pages in search results. The algorithm works by assigning a numerical score to each page based on the number and quality of other pages that link to it. The more links a page has from other high-quality pages, the higher its PageRank score.\n\nIn the Tweepcred project, the PageRank algorithm is used to determine the influence of Twitter users based on their interactions with other users. The graph is constructed by treating Twitter users as nodes, and their interactions (mentions, retweets, etc.) as edges. The PageRank score of a user represents their influence in the network.', metadata={'source': './the-algorith

if you run `compute_engine` on a local datasets it will outperform `python` implementation significantly.

### Creating dataset on AWS S3

In [82]:
dataset_path = f"s3://BUCKET/langchain_test" # could be also ./local/path (much faster locally), hub://bucket/path/to/dataset, gcs://path/to/dataset, etc.

embedding = OpenAIEmbeddings()
db = DeepLake.from_documents(docs, dataset_path=dataset_path, embedding=embeddings, overwrite=True, creds = {
   'aws_access_key_id': os.environ['AWS_ACCESS_KEY_ID'], 
   'aws_secret_access_key':  os.environ['AWS_SECRET_ACCESS_KEY'], 
   'aws_session_token': os.environ['AWS_SESSION_TOKEN'], # Optional
})

s3://hub-2.0-datasets-n/langchain_test loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:10<00:00
\

Dataset(path='s3://hub-2.0-datasets-n/langchain_test', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (4, 1536)  float32   None   
    ids      text     (4, 1)      str     None   
 metadata    json     (4, 1)      str     None   
   text      text     (4, 1)      str     None   


## Deep Lake API
you can access the Deep Lake  dataset at `db.ds`

In [23]:
# get structure of the dataset
db.deeplake_vector_store.dataset.summary()

Dataset(path='./my_deeplake/', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    ids       text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


In [24]:
# get embeddings numpy array
embeds = db.deeplake_vector_store.dataset.embedding.numpy()

### Transfer local dataset to cloud
Copy already created dataset to the cloud. You can also transfer from cloud to local.

In [73]:
import deeplake
username = "davitbun" # your username on app.activeloop.ai   
source = f"hub://{username}/langchain_test" # could be local, s3, gcs, etc.
destination = f"hub://{username}/langchain_test_copy" # could be local, s3, gcs, etc.

deeplake.deepcopy(src=source, dest=destination, overwrite=True)


Copying dataset: 100%|██████████| 56/56 [00:38<00:00


This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/davitbun/langchain_test_copy
Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


Dataset(path='hub://davitbun/langchain_test_copy', tensors=['embedding', 'ids', 'metadata', 'text'])

In [76]:
db = DeepLake(dataset_path=destination, embedding_function=embeddings)
db.add_documents(docs)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/davitbun/langchain_test_copy



/

hub://davitbun/langchain_test_copy loaded successfully.



Deep Lake Dataset in hub://davitbun/langchain_test_copy already exists, loading from the storage


Dataset(path='hub://davitbun/langchain_test_copy', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (4, 1536)  float32   None   
    ids      text     (4, 1)      str     None   
 metadata    json     (4, 1)      str     None   
   text      text     (4, 1)      str     None   


Evaluating ingest: 100%|██████████| 1/1 [00:31<00:00
-

Dataset(path='hub://davitbun/langchain_test_copy', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (8, 1536)  float32   None   
    ids      text     (8, 1)      str     None   
 metadata    json     (8, 1)      str     None   
   text      text     (8, 1)      str     None   


['ad42f3fe-e188-11ed-b66d-41c5f7b85421',
 'ad42f3ff-e188-11ed-b66d-41c5f7b85421',
 'ad42f400-e188-11ed-b66d-41c5f7b85421',
 'ad42f401-e188-11ed-b66d-41c5f7b85421']